subjectlist 처음뽑고, 삽관발관 데이터 뽑기

In [1]:
import psycopg2
from dfply import *
import pandas as pd
import logging
import sys
import os
from pathlib import Path
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine

# 현재 노트북 파일의 상위 디렉토리에 있는 src 디렉토리의 경로를 sys.path에 추가
module_path = Path('../src').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Logging config
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 소스코드(src)
import src.data_extraction.access_database as db
import src.data_extraction.filter_adult_patients as fap
import src.data_extraction.filter_ventilation_events as fve
from src.utils import utils


### 1. DB 연결

#### 1-1. 각종 파라미터 설정

In [2]:
# 데이터 저장위치
output_dir = './data'

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

# 호흡기 ITEM ID 설정 (웬만하면 건들지 마세요)
vent_ids_config = {
    'INTUBATION_ITEM_IDS': "224385",
    'EXTUBATION_ITEM_IDS': "225468, 225477, 227194"
}

# 테이블 쿼리 설정
tables_query = {
    'pg_tables': 'SELECT * FROM PG_TABLES;',
    'patients': 'SELECT * FROM mimiciv_hosp.patients;',
    'admissions': 'SELECT * FROM mimiciv_hosp.admissions;',
    'transfers': 'SELECT * FROM mimiciv_hosp.transfers;',
    'icustays': 'SELECT * FROM mimiciv_icu.icustays;',
    'd_items': 'SELECT * FROM mimiciv_icu.d_items;',
    'intubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['INTUBATION_ITEM_IDS']});",
    'extubation': f"SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN ({vent_ids_config['EXTUBATION_ITEM_IDS']});",
    'ventilation': 'SELECT * FROM mimiciv_derived.ventilation;'
}
# 추가 설정 (웬만하면 건들지 마세요)
additional_config = {
    'TIME_DIFF_DUP': 30   # time_diff_dup: 몇 분 이내에 삽관/발관 이벤트를 중복으로 정의할 것인가? (디폴트: 30분)
}

db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_onlyselect
Password: *************
Host: 1.212.63.162
Port: 35430

--------- SQL Queries for Required Tables ---------
pg_tables: SELECT * FROM PG_TABLES;
patients: SELECT * FROM mimiciv_hosp.patients;
admissions: SELECT * FROM mimiciv_hosp.admissions;
transfers: SELECT * FROM mimiciv_hosp.transfers;
icustays: SELECT * FROM mimiciv_icu.icustays;
d_items: SELECT * FROM mimiciv_icu.d_items;
intubation: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (224385);
extubation: SELECT * FROM mimiciv_icu.procedureevents WHERE itemid IN (225468, 225477, 227194);
ventilation: SELECT * FROM mimiciv_derived.ventilation;



#### 1-2. DB 연결, 데이터 추출

In [3]:
import psycopg2

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

try:
    # 데이터베이스 연결
    conn = psycopg2.connect(**db_config)
    print("Connected to the database successfully!")

    # 커서 생성
    cur = conn.cursor()

    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")

    # SQL 쿼리 실행
    cur.execute("SELECT version();")

    # 결과 가져오기
    result = cur.fetchone()
    print("PostgreSQL version:", result)

except (psycopg2.Error, Exception) as error:
    print("Error while connecting to the database:", error)


# finally:
#     # 커서와 연결 종료
#     if 'cur' in locals():
#         cur.close()
#     if 'conn' in locals():
#         conn.close()
#     print("Database connection closed.")

Connected to the database successfully!


2024-07-18 16:23:05,606 - INFO - Retrieved pg_tables: (164, 8)
2024-07-18 16:23:13,543 - INFO - Retrieved patients: (299712, 6)
2024-07-18 16:23:35,864 - INFO - Retrieved admissions: (431231, 16)
2024-07-18 16:24:19,426 - INFO - Retrieved transfers: (1890972, 7)
2024-07-18 16:24:21,380 - INFO - Retrieved icustays: (73181, 8)
2024-07-18 16:24:21,439 - INFO - Retrieved d_items: (4014, 9)
2024-07-18 16:24:21,695 - INFO - Retrieved intubation: (8488, 22)
2024-07-18 16:24:22,591 - INFO - Retrieved extubation: (23122, 22)
2024-07-18 16:24:23,979 - INFO - Retrieved ventilation: (109200, 4)


PostgreSQL version: ('PostgreSQL 15.7 (Ubuntu 15.7-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [4]:
dataframes.keys()   # 메모리에 저장된 데이터 확인

dict_keys(['pg_tables', 'patients', 'admissions', 'transfers', 'icustays', 'd_items', 'intubation', 'extubation', 'ventilation'])

In [5]:
# 데이터프레임 변환
patients = dataframes['patients']
admissions = dataframes['admissions']
intubation_all = dataframes['intubation']
extubation_all = dataframes['extubation']
icustays = dataframes['icustays']
ventilation = dataframes['ventilation']

print(f'patients: {patients.shape}')
print(f'admissions: {admissions.shape}')
print(f'intubation_all: {intubation_all.shape}')
print(f'extubation_all: {extubation_all.shape}')
print(f'icustays: {icustays.shape}')
print(f'ventilation: {ventilation.shape}')

patients: (299712, 6)
admissions: (431231, 16)
intubation_all: (8488, 22)
extubation_all: (23122, 22)
icustays: (73181, 8)
ventilation: (109200, 4)


### 2. 데이터 처리
- 2-1. 환자 정보 정제
- 2-2. 호흡기(삽관/발관) 정보 정제

#### 2.1. 환자 정보 필터링
- 성인 환자 (anchor age >= 18) >>

    - 병원 입원(hadm_id) 정보가 있는 환자 >>
    
        -  중환자실 입원(stay_id) 정보가 있는 환자

In [6]:
## 데이터 처리: 응급병동 환자 데이터 (filter_adult_patients)
# 성인환자 데이터 필터링
adults_pat = fap.filter_adult_patients(patients)   # 18세 이상 필터링
adults_hadm = fap.merge_patient_admissions(adults_pat, admissions)   # patient, admissions 테이블 결합
adults_hadm = fap.remove_missing_hadm(adults_hadm)   # 입원정보(hadm_id) 없는 행 삭제
adults_icu = fap.merge_with_icu(adults_hadm, icustays)   # icu (응급병동) 테이블 결합
adults_icu = fap.remove_missing_icu_stays(adults_icu)   # icu 입원정보(stay_id) 없는 행 삭제

Number of adult patients retrieved: 299712


#### 2.2. 삽관/발관 데이터 정제
- 먼저 삽관 테이블, 발관 테이블 따로 처리 (함수: filter_and_label_ventilation_data, filter_close_events)
    - 필요한 칼럼 가져오기: "subject_id", "hadm_id", "stay_id", "starttime", "itemid", "patientweight"
    - (발관 테이블) extubation cause 라벨 붙여주기
    - 근접행 제거하기 (additional_config 변수의 'TIME_DIFF_DUP' 파라미터 값 참조)
- 삽관/발관 테이블 결합해주기 (함수: join_ventilation_and_rename, join_admissions)
    - 중복되는 이름 변경해주기
    - 환자 정보와 결합해주기

In [7]:
## 데이터 처리: 삽관/발관 데이터 (filter_ventilation_events)
# 삽관/발관 데이터 필터링 및 처리
intubation_data = fve.filter_and_label_ventilation_data(intubation_all, 'intubationtime', 'intubation')
extubation_data = fve.filter_and_label_ventilation_data(extubation_all, 'extubationtime', 'extubation')

# 근접행 제거: time_diff(분) 이내
time_diff = additional_config["TIME_DIFF_DUP"]
intubation_data = fve.filter_close_events(intubation_data, 'intubationtime', ['subject_id', 'hadm_id'], time_diff=time_diff)
extubation_data = fve.filter_close_events(extubation_data, 'extubationtime', ['subject_id', 'hadm_id'], time_diff=time_diff)

# 삽관 발관 테이블 결합
intubation_extubation = fve.join_ventilation_and_rename(intubation_data, extubation_data)

# 입원 데이터 결합
intubation_extubation = fve.join_admissions(intubation_extubation, admissions)


# 중복치/근접치 제거 리포트 출력
fve.report_filtering_stats('intubation', intubation_all, intubation_data, time_diff)
fve.report_filtering_stats('extubation', extubation_all, extubation_data, time_diff)

----------------------------------------------------------------------
intubation 중복치 및 근접치 통계
근접한 행 과의 거리가 30분 이내인 값을 중복행으로 정의
제거된 intubation 중복 행 개수 : 223 행 / 8488 행 (2.63 %)
----------------------------------------------------------------------
extubation 중복치 및 근접치 통계
근접한 행 과의 거리가 30분 이내인 값을 중복행으로 정의
제거된 extubation 중복 행 개수 : 440 행 / 23122 행 (1.90 %)


### 데이터 저장

In [8]:
# 데이터 저장
if not os.path.exists(output_dir):   # output 디렉토리가 없을 경우 생성
    os.makedirs(output_dir)

utils.save_filtered_data(adults_icu, intubation_extubation, output_dir, outputs='all')

intubation_data.to_csv('./data/intubation_data.csv')
extubation_data.to_csv('./data/extubation_data.csv')
ventilation.to_csv('./data/ventilation.csv')

Data extraction and processing complete. Files saved.
Data extraction and processing complete. Files saved.
